In [1]:
import numpy as np
import pandas as pd

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression

from sklearn import metrics

In [2]:
df = pd.read_csv("./Absenteeism-data.csv")

In [3]:
df = df.drop(['ID'], axis = 1)

In [4]:
reason_columns = pd.get_dummies(df['Reason for Absence'], drop_first = True)

reason_type_1 = reason_columns.loc[:, 1:14].max(axis=1)
reason_type_2 = reason_columns.loc[:, 15:17].max(axis=1)
reason_type_3 = reason_columns.loc[:, 18:21].max(axis=1)
reason_type_4 = reason_columns.loc[:, 22:].max(axis=1)

df = df.drop(['Reason for Absence'], axis = 1)

df = pd.concat([df, reason_type_1, reason_type_2, reason_type_3, reason_type_4], axis = 1)

In [5]:
df['Date'] = pd.to_datetime(df['Date'], format = '%d/%m/%Y')

list_months = []
for i in range(df.shape[0]):
    list_months.append(df['Date'][i].month)
df['Month Value'] = list_months

def date_to_weekday(date_value):
    return date_value.weekday()
df['Day of the Week'] = df['Date'].apply(date_to_weekday)

df = df.drop(['Date'], axis = 1)

In [6]:
df['Education'] = df['Education'].map({1:0, 2:1, 3:1, 4:1})

In [7]:
# parameterized code
df['Excessive Absenteeism'] = np.where(df['Absenteeism Time in Hours'] > 
                                       df['Absenteeism Time in Hours'].median(), 1, 0)
df = df.drop(['Absenteeism Time in Hours'],axis=1)

In [8]:
df.head(1)

,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,0,1,2,3,Month Value,Day of the Week,Excessive Absenteeism
0,289,36,33,239.554,30,0,2,1,0,0,0,1,7,1,1


In [9]:
# create a variable that will contain the inputs (everything without the targets)
unscaled_inputs = df.drop(['Excessive Absenteeism'], axis=1)  #df.iloc[:,:-1]
targets = df['Excessive Absenteeism']

In [10]:
class CustomScaler(BaseEstimator,TransformerMixin): 
    def __init__(self,columns,copy=True,with_mean=True,with_std=True):
        self.scaler = StandardScaler(copy,with_mean,with_std)
        self.columns = columns
        self.mean_ = None
        self.var_ = None

    def fit(self, X, y=None):
        self.scaler.fit(X[self.columns], y)
        self.mean_ = np.mean(X[self.columns])
        self.var_ = np.var(X[self.columns])
        return self

    def transform(self, X, y=None, copy=None):
        # record the initial order of the columns
        init_col_order = X.columns
        
        # scale all features that you chose when creating the instance of the class
        X_scaled = self.scaler.transform(X[self.columns])
        X_scaled = pd.DataFrame(X_scaled, columns=self.columns)
        
        # declare a variable containing all information that was not scaled
        X_not_scaled = X.loc[:,~X.columns.isin(self.columns)]
        
        # return a data frame which contains all scaled features and all 'not scaled' features
        return pd.concat([X_not_scaled, X_scaled], axis=1)
    
        # use the original order (that you recorded in the beginning)
        #return pd.concat([X_not_scaled, X_scaled], axis=1)[init_col_order]

In [11]:
# Traditional Approach
# scaled_inputs = StandardScaler().fit_transform(unscaled_inputs)

columns_to_omit = [0, 1, 2, 3,'Education']
columns_to_scale = [x for x in unscaled_inputs.columns.values if x not in columns_to_omit]
absenteeism_scaler = CustomScaler(columns_to_scale)
absenteeism_scaler.fit(unscaled_inputs)
scaled_inputs = absenteeism_scaler.transform(unscaled_inputs)

In [12]:
scaled_inputs

,Education,0,1,2,3,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Children,Pets,Month Value,Day of the Week
0,0,0,0,0,1,1.005844,0.412816,-0.536062,-0.806331,0.767431,0.880469,0.268487,0.182726,-0.683704
1,0,0,0,0,0,-1.574681,-1.141882,2.130803,-0.806331,1.002633,-0.019280,-0.589690,0.182726,-0.683704
2,0,0,0,0,1,-0.654143,1.426749,0.248310,-0.806331,1.002633,-0.919030,-0.589690,0.182726,-0.007725
3,0,1,0,0,0,0.854936,-1.682647,0.405184,-0.806331,-0.643782,0.880469,-0.589690,0.182726,0.668253
4,0,0,0,0,1,1.005844,0.412816,-0.536062,-0.806331,0.767431,0.880469,0.268487,0.182726,0.668253
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
695,1,1,0,0,0,-0.654143,-0.533522,0.562059,-0.853789,-1.114186,0.880469,-0.589690,-0.388293,-0.007725
696,0,1,0,0,0,0.040034,-0.263140,-1.320435,-0.853789,-0.643782,-0.019280,1.126663,-0.388293,-0.007725
697,1,1,0,0,0,1.624567,-0.939096,-1.320435,-0.853789,-0.408580,-0.919030,-0.589690,-0.388293,0.668253
698,1,0,0,0,1,0.190942,-0.939096,-0.692937,-0.853789,-0.408580,-0.919030,-0.589690,-0.388293,0.668253


In [13]:
x_train, x_test, y_train, y_test = train_test_split(scaled_inputs, targets,
                                                    test_size = 0.2, random_state = 20)

In [14]:
# create a logistic regression object
reg = LogisticRegression().fit(x_train,y_train)

print(reg.score(x_train,y_train))
print(reg.score(x_test,y_test))

0.775
0.7428571428571429


In [15]:
import pickle

with open('model', 'wb') as file:
    pickle.dump(reg, file)
    
with open('scaler','wb') as file:
    pickle.dump(absenteeism_scaler, file)